In [1]:
# Simplified data loading (unsupervised)
from pathlib import Path

import pandas as pd

# Paths
DATA_DIR = Path("..") / "data" / "model_ready"
CSV = DATA_DIR / "cpt_features.csv"

# Load numeric-only dataset for unsupervised learning
df = pd.read_csv(CSV)
X = df.copy()  # all columns are numeric features

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
X.head(3)

Dataset loaded: 2345409 rows, 9 columns


,Depth (m),qc (MPa),fs (kPa),Rf (%),"σ,v (kPa)",u0 (kPa),"σ',v (kPa)",Qtn (-),Fr (%)
0,0.02,1.15,1.0,0.06,0.38,0.20,0.18,8.04,0.06
1,0.04,2.85,1.0,0.03,0.76,0.39,0.37,185.53,0.03
2,0.06,4.80,1.0,0.06,1.14,0.59,0.55,205.79,0.06


# Unsupervised Clustering in Geotechnics: K-Means and HDBSCAN

This notebook introduces two popular clustering techniques and applies them to simple, geotechnics-inspired datasets:

- K-Means: partition data into k clusters; fast and simple, but requires choosing k and assumes roughly spherical clusters.
- HDBSCAN: density-based clustering; automatically finds the number of clusters, handles noise/outliers, works with variable density.

We'll start with a small synthetic dataset (toy CPT/soil-behavior-inspired) and then optionally try real data from `data/raw/earthquake_data.csv` and `data/raw/CPT_PremstallerGeotechnik_revised.csv` (CPT measurements).

Learning goals:
- Understand the intuition behind K-Means and HDBSCAN
- Practice feature scaling, dimensionality reduction for visualization
- Evaluate clustering via silhouette score and qualitative plots
- See pros/cons in geotechnical contexts (e.g., soil behavior zones, mixed stratigraphy, noise/outliers)

## Setup
We import libraries used throughout. If you run this with the project environment (uv), dependencies are already declared in `pyproject.toml` (including `hdbscan`).

In [ ]:
# Imports

# HDBSCAN
import hdbscan
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Ensure plots render inline
%matplotlib inline

## Quick theory recap (teacher notes)

- K-Means minimizes within-cluster variance. Sensitive to scale and initialization; choose k via elbow or silhouette.
- HDBSCAN builds a hierarchy of clusters from density estimation; extracts stable clusters and labels sparse points as noise (−1).
- In geotechnics: cluster CPT features (e.g., qc, fs, Fr, Qtn) to reveal soil behavior zones or facies patterns, but beware of depth trends and heteroscedasticity.

## 1) Synthetic geotechnical-style dataset
We'll create a toy dataset mimicking two soil layers plus a mixed transition:
- Features: depth (m), normalized cone resistance qc (MPa), friction ratio Fr (%).
- Lower depth tends to be dense sand (higher qc, low Fr), upper is softer clay/silt (lower qc, higher Fr).
This is simplified and only for teaching clustering behavior.

In [ ]:
rng = np.random.default_rng(42)
n = 600
depth = np.sort(rng.uniform(0, 20, size=n))  # 0-20 m

# Synthetic behaviors: upper 0-8 m mostly softer (low qc, higher Fr); 8-20 m denser (high qc, lower Fr)
qc = np.where(
    depth < 8,
    rng.normal(5, 1.0, size=n),  # MPa (toy numbers)
    rng.normal(15, 2.0, size=n),
)
Fr = np.where(
    depth < 8,
    rng.normal(3.0, 0.8, size=n),  # %
    rng.normal(1.0, 0.3, size=n),
)
# Add a transition zone scatter
mask = (depth > 6) & (depth < 10)
qc[mask] += rng.normal(0, 3.0, size=mask.sum())
Fr[mask] += rng.normal(0, 0.6, size=mask.sum())

df_syn = pd.DataFrame({"depth_m": depth, "qc_mpa": qc, "Fr_pct": Fr})
df_syn.head()

In [ ]:
fig = px.scatter(
    df_syn,
    x="qc_mpa",
    y="depth_m",
    color="Fr_pct",
    title="Synthetic CPT-like Data: qc vs depth (color = Fr)",
    labels={"qc_mpa": "qc (MPa)", "depth_m": "Depth (m)", "Fr_pct": "Fr (%)"},
    color_continuous_scale="Viridis",
    height=500,
)
fig.update_yaxes(autorange="reversed")  # Depth increases downward
fig.show()

## 2) K-Means clustering (start simple)
We'll cluster on standardized features [qc, Fr, depth].

### Visualize synthetic data
Depth vs qc and Fr. We expect a vertical trend due to layering.

In [ ]:
features = df_syn[["qc_mpa", "Fr_pct", "depth_m"]].copy()
scaler = StandardScaler()
X = scaler.fit_transform(features)

# Try k from 2 to 6, compute inertia and silhouette
inertias, sils, ks = [], [], []
for k in range(2, 7):
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X)
    ks.append(k)
    inertias.append(km.inertia_)
    sils.append(silhouette_score(X, labels))

pd.DataFrame({"k": ks, "inertia": inertias, "silhouette": sils})

In [ ]:
fig = px.line(
    x=ks,
    y=inertias,
    markers=True,
    title="K-Means: Elbow (Inertia)",
    labels={"x": "k", "y": "Inertia"},
)
fig.show()
fig2 = px.line(
    x=ks,
    y=sils,
    markers=True,
    title="K-Means: Silhouette vs k",
    labels={"x": "k", "y": "Silhouette"},
)
fig2.show()

Pick k based on elbow and silhouette (often k=2 or 3 here). Then fit and visualize in depth space and in a PCA projection for separation intuition.

In [ ]:
best_k = int(pd.Series(sils, index=ks).idxmax())  # simple choice for demo
km = KMeans(n_clusters=best_k, n_init=10, random_state=42)
labels_km = km.fit_predict(X)
df_syn["cluster_kmeans"] = labels_km.astype(int)

# Visualize along depth
fig = px.scatter(
    df_syn,
    x="qc_mpa",
    y="depth_m",
    color="cluster_kmeans",
    title=f"K-Means (k={best_k}) clusters in qc-depth space",
    labels={"qc_mpa": "qc (MPa)", "depth_m": "Depth (m)"},
    height=500,
)
fig.update_yaxes(autorange="reversed")
fig.show()

# PCA for 2D viz
pca = PCA(n_components=2, random_state=42)
X2 = pca.fit_transform(X)
df_plot = pd.DataFrame(X2, columns=["PC1", "PC2"])
df_plot["cluster"] = labels_km.astype(int)
fig2 = px.scatter(
    df_plot, x="PC1", y="PC2", color="cluster", title="K-Means clusters in PCA space"
)
fig2.show()

## 3) HDBSCAN clustering
HDBSCAN doesn't need k; we tune `min_cluster_size` to control granularity. It can label noise as −1.

In [ ]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=25, min_samples=None, cluster_selection_epsilon=0.0
)
labels_hdb = clusterer.fit_predict(X)
df_syn["cluster_hdbscan"] = labels_hdb.astype(int)
n_noise = int((labels_hdb == -1).sum())
n_clusters = int(len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0))
print({"n_clusters": n_clusters, "n_noise": n_noise})

### Discussion prompts
- Which method better separates the two layers and the transition zone?
- What happens if you remove depth from features?
- How does `min_cluster_size` affect HDBSCAN?
- When would you prefer silhouette evaluation vs density-based approaches in practice?

In [ ]:
fig = px.scatter(
    df_syn,
    x="qc_mpa",
    y="depth_m",
    color="cluster_hdbscan",
    title="HDBSCAN clusters in qc-depth space (−1=noise)",
    labels={"qc_mpa": "qc (MPa)", "depth_m": "Depth (m)"},
    height=500,
)
fig.update_yaxes(autorange="reversed")
fig.show()

# PCA viz
_df_plot = pd.DataFrame(X2, columns=["PC1", "PC2"])  # reuse PCA from above
_df_plot["cluster"] = labels_hdb.astype(int)
fig2 = px.scatter(
    _df_plot,
    x="PC1",
    y="PC2",
    color="cluster",
    title="HDBSCAN clusters in PCA space (−1=noise)",
)
fig2.show()